<a href="https://colab.research.google.com/github/Sakibapon/NLP/blob/master/Fake_News_Tfidf_Multinomial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
ps=PorterStemmer()
lm=WordNetLemmatizer()

In [2]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/Dataset/Train.Fake_News.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
x=df.drop('label',axis=1)
y=df['label']
x.shape,y.shape

((20800, 4), (20800,))

In [4]:
df=df.dropna()


In [5]:
msg=df.copy()
msg.reset_index(inplace=True)
msg.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer

In [8]:
corpus=[]
for i in range(len(msg)):
  words=re.sub('[^a-zA-Z]',' ',msg['title'][i])
  words=words.lower()
  words=words.split()
  words=[ps.stem(word)for word in words if word not in stopwords.words('english')]
  words=' '.join(words)
  corpus.append(words)

In [9]:
tf=TfidfVectorizer(max_features=5000,ngram_range=(1,3))
x=tf.fit_transform(corpus).toarray()

In [21]:
pdf=pd.DataFrame(x,columns=tf.get_feature_names())
pdf.head()

,aaron,ab,abandon,abba,abc,abc news,abduct,abe,abedin,abl,aboard,abort,abroad,absolut,abstain,abstain un,abstain un vote,absurd,abu,abus,abus new,abus new york,academ,academi,accept,access,access pipelin,access pipelin protest,accid,accident,accident post,accord,account,accus,accus trump,achiev,acid,acknowledg,acknowledg emf,acknowledg emf damag,...,year eve,year later,year new,year new york,year old,year old girl,year sinc,year war,yellen,yemen,yet,yet anoth,yiannopoulo,yield,yo,york,york citi,york new,york new york,york state,york time,yorker,young,young peopl,youth,youtub,yr,zealand,zero,zika,zika new,zika new york,zika viru,zionist,zone,zone new,zone new york,zoo,zu,zuckerberg
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
y=msg['label']
x.shape,y.shape

((18285, 5000), (18285,))

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.33,random_state=0)

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,accuracy_score
mb=MultinomialNB()
mb.fit(X_train,y_train)
pr=mb.predict(X_test)
score=accuracy_score(y_test,pr)
print(score)
print(classification_report(y_test,pr))

0.8810273405136703
              precision    recall  f1-score   support

           0       0.86      0.94      0.90      3389
           1       0.91      0.80      0.86      2646

    accuracy                           0.88      6035
   macro avg       0.89      0.87      0.88      6035
weighted avg       0.88      0.88      0.88      6035



In [13]:
from sklearn.linear_model import PassiveAggressiveClassifier
pav=PassiveAggressiveClassifier()
pav.fit(X_train,y_train)
pred=pav.predict(X_test)
score=accuracy_score(y_test,pred)
print(score)
print(classification_report(y_test,pred))

0.9178127589063795
              precision    recall  f1-score   support

           0       0.93      0.92      0.93      3389
           1       0.90      0.92      0.91      2646

    accuracy                           0.92      6035
   macro avg       0.92      0.92      0.92      6035
weighted avg       0.92      0.92      0.92      6035



In [15]:
cv=MultinomialNB(alpha=0.1)
bests=0
for alpha in np.arange(0,1,0.1):
  mcv=MultinomialNB(alpha=alpha)
  mcv.fit(X_train,y_train)
  predic=mcv.predict(X_test)
  scores=accuracy_score(y_test,predic)
  if scores>bests:
    cv=mcv
    print('Alpha is {} and Score= {}'.format(alpha,scores))

/usr/local/lib/python3.6/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Alpha is 0.0 and Score= 0.8662800331400166
Alpha is 0.1 and Score= 0.8777133388566695
Alpha is 0.2 and Score= 0.8801988400994201
Alpha is 0.30000000000000004 and Score= 0.87986743993372
Alpha is 0.4 and Score= 0.8808616404308203
Alpha is 0.5 and Score= 0.8806959403479702
Alpha is 0.6000000000000001 and Score= 0.8815244407622204
Alpha is 0.7000000000000001 and Score= 0.8813587406793704
Alpha is 0.8 and Score= 0.8816901408450705
Alpha is 0.9 and Score= 0.8816901408450705
